MS peak align based on bioAssay peak alignment for excipient screening
see nb-dejejord-5094579-001

In [56]:
#Loading Modules
import pandas as pd #handling of csv files and dataframes
import numpy as np

In [57]:
#loading data 
data = pd.read_excel('./exampledata/Week2_Trimmed.xlsx')


pre-processing and qc steps
return parent peak retention time median, min RT, max RT, enter global parent peak RT (optional)
parent API peak cutoff


In [58]:
#split vial column into before and after the colon into plate, and Vial columns
df_vial_counts = data.groupby(['Vial']).size().reset_index(name='peakCount')
maxarea = data.groupby(['Vial'])['Area'].max().reset_index()
areasum = data.groupby(['Vial'])['Area'].sum().reset_index(name='Sum of Area')
maxarea['Area Sum'] = areasum['Sum of Area']
maxarea['Area Ratio'] = maxarea['Area']/ maxarea['Area Sum']
RTmax= (data.sort_values(['Vial', 'Area'], ascending=[True, False])
             .drop_duplicates(['Vial']).reset_index(drop=True)
          )
maxarea = pd.merge(maxarea,RTmax,how = 'left', on = ['Vial', 'Area'])
maxarea = pd.merge(maxarea,df_vial_counts, how = 'left', on = 'Vial')


In [59]:

median_retention_time = maxarea['Retention Time'].median()
min_retention_time = maxarea['Retention Time'].min()
max_retention_time = maxarea['Retention Time'].max()


In [60]:
# Group the data by "Vial" and find the index of the row with the maximum "Area" for each group
max_area_indices = data.groupby('Vial')['Area'].idxmax()
print(max_area_indices.shape)

(144,)


In [61]:
# run the msalign function with the retention times
aligned_array = msalign(x,array, retention_times, weights=[100,100], only_shift=False)


NameError: name 'msalign' is not defined

In [ ]:
# start by storing file locally in my local directory
# once we have this working, we can replace this with asking the user to upload the files
#focus on saving the output to the same directory as the input file


I am in favor of getting rid of what's below 

In [ ]:
# This part was an attempt of having an interface to click and select files 
#It does currently not work

from tk import  Checkbutton, Button, IntVar

def get_selected_items():
    global csv_file_path
    selected_items = [item.get() for item in checkbox_vars]
    csv_file_path = []
    print("Selected items:", selected_items)
    for j,counter in enumerate(selected_items):
        if counter==True:
            csv_file_path.append(filedialog.askopenfilename(title="Select CSV File for ", filetypes=[("CSV Files", "*.csv")]))
            # Display the selected file path
            print("Selected CSV File for ",checkbox_names[j], " is: ", csv_file_path[j])
        else: csv_file_path.append("0")
    root.quit()

# Create the Tkinter root window
root = Tk()

# Create tkinter IntVar variables to store the state of the checkboxes
checkbox_vars = [IntVar() for _ in range(6)]
checkbox_names = ["Week0","Week0_humid","Week2","Week2_humid","Week5","Week5_humid"]

# Create the checkboxes
checkboxes = []
for i, var in enumerate(checkbox_vars):
    checkbox = Checkbutton(root, text=checkbox_names[i], variable=var)
    checkbox.pack()
    checkboxes.append(checkbox)

# Create a button to get the selected items
button = Button(root, text="Get Selected Items", command=get_selected_items)
button.pack()

# Run the Tkinter event loop
root.mainloop()

In [ ]:
#Loading of xlsx-files and reformatting of dataframe - Week2 loaded as example
df = pd.read_excel("C:/Users/dejejord/OneDrive - Merck Sharp & Dohme LLC/Desktop/ExcipientCompatibility_Jupyter/Week2_Trimmed.xlsx")

#Trim down to most important columns
df1 = df[["Vial","Retention Time", "Area"]]

#Get relative retention times by division by retention time of largest area peak ==> Caveat: largest area needs to be API peak
max_area = df1["Area"].max()
max_area_row=df1[df1["Area"] == max_area]
max_Ret_value = max_area_row["Retention Time"].iloc[0]
df1["Retention Time"] = df1["Retention Time"] / max_Ret_value

#Pivot dataframe to be sorted by retention times
df1=df1.round(2)
pivot_df1 = df1.pivot_table(index="Vial", columns="Retention Time", values="Area",aggfunc='sum')
pivot_df1[np.isnan(pivot_df1)] = 0

#Merge relative retention times if difference is 0.01 and no entry recorded for one of two columns for each well
df_merged = pd.DataFrame()
col_del = []
columns = pivot_df1.columns.tolist()
c=0

while c <= len(columns)-2:
    merge_condition = False
    for index,row in pivot_df1.iterrows():
        if round((columns[c+1]-columns[c]),2)==0.01:
            if (row[columns[c+1]] != 0.0 and row[columns[c]] == 0.0) or (row[columns[c+1]] == 0.0 and row[columns[c]] != 0.0):
                merge_condition = True
                df_merged[columns[c]] =  pivot_df1[columns[c]]+pivot_df1[columns[c+1]]
                c=c+2
                break
    if merge_condition == False:
        df_merged[columns[c]] =  pivot_df1[columns[c]]
        c=c+1
if c == len(columns)-1:
    df_merged[columns[c]] =  pivot_df1[columns[c]]


#Normalize by API peak area
final_norm = df_merged.div(df_merged[df_merged.columns[df_merged.columns == 0.99]].values,axis=0)
final_norm=final_norm.round(5)

In [ ]:
df.set_index(["Vial"])["SampleName"]

In [ ]:
non_zero_counts = np.count_nonzero(df_merged, axis=0)
result = pd.Series(non_zero_counts, index=df_merged.columns)

print(result)

In [ ]:
final_norm